For each geometry:
* combine output files from geometry generation and simulation into one pickle file with consistent conventions (e.g., all 0-indexing). Remove cases with overlapping edges.
* create a .png with a visualization of the microstructure.

# Imports & path definitions

In [ ]:
import pickle
import numpy as np
import os
import scipy.io as sio
import funcs_helpers as fh
import matplotlib.pyplot as plt
import helper_funcs as mf

In [ ]:
results_path = r"E:\OneDrive - TU Eindhoven\Werk\matlab\Martins_code\data"

geometries_path = r"E:\OneDrive - TU Eindhoven\Werk\python\wallpaper_dataset\data\dataset1\simulated"

final_path = r"E:\OneDrive - TU Eindhoven\Werk\python\wallpaper_dataset\data\dataset1\final_dataset\pickle_files"

# create final path if it does not exist
if not os.path.exists(final_path):
    os.makedirs(final_path)

geoms = [f for f in os.listdir(geometries_path)]
print(f'{len(geoms)} geometries found')

# reruns of errorFlag trajs with perturbation error fixed
reruns_path = r"E:\OneDrive - TU Eindhoven\Werk\matlab\Martins_code\data_reruns"

# reruns of remaining errorFlag trajs with smaller minimum time step
reruns2_path = r"E:\OneDrive - TU Eindhoven\Werk\matlab\Martins_code\data_reruns2"

# reruns of too short trajs with smaller default time step
reruns3_path = r"E:\OneDrive - TU Eindhoven\Werk\matlab\Martins_code\data_reruns3"

reruns = [f for f in os.listdir(reruns_path) if f.endswith('.mat')]
reruns2 = [f for f in os.listdir(reruns2_path) if f.endswith('.mat')]
reruns3 = [f for f in os.listdir(reruns3_path) if f.endswith('.mat')]

refD_path = r'E:\OneDrive - TU Eindhoven\Werk\matlab\Martins_code\data_refD'

plot_contact = True

# Check if set of simulated geometries exactly matches the set of generated geometries

In [ ]:
files1 = [f for f in os.listdir(geometries_path)]
print('Nr of geometries:', len(files1))
files2 = [f[:-4] for f in os.listdir(results_path) if f.endswith(".mat") and not f.endswith("specialnodes.mat")]
print('Nr of results:', len(files2))
files_specnodes = [f[:-4] for f in os.listdir(results_path) if f.endswith("specialnodes.mat")]
print('Nr of special nodes files:', len(files_specnodes))
files_specnodes2 = [f[:-17] for f in os.listdir(results_path) if f.endswith("specialnodes.mat")]

# check that for each geometry there is one result file and one special nodes file
assert np.all(np.sort(files1) == np.sort(files2))
assert np.all(np.sort(files1) == np.sort(files_specnodes2))

# Iterate over all geometries

In [ ]:
errors = []
rerun_found = False
for g, geom in enumerate(geoms):
    print(f'======= {g}, {geom} =======')
    # ====================== LOAD SIMULATION DATA ======================
    data = {'simulations': {'trajectories': {}, 'time_steps': {}},
            'geometry': {'fundamental_domain': {}, 'unit_cell': {}},
            'mesh': {'fundamental_domain': {}, 'unit_cell': {}, 'RVE': {}}
            }
    matfile = os.path.join(results_path, geom + '.mat')
    data_from_mat = sio.loadmat(matfile)
    for key in data_from_mat['data_sim'].dtype.names:
        data['simulations']['trajectories'][key] = data_from_mat['data_sim'][key][0, 0]
    for key in data_from_mat['data_ts'].dtype.names:
        data['simulations']['time_steps'][key] = data_from_mat['data_ts'][key][0, 0]

    # change from Matlab 1-indexing to Python 0-indexing
    data['simulations']['time_steps']['traj'] -= 1

    # reshuffle P
    data['simulations']['time_steps']['P'] = data['simulations']['time_steps']['P'].reshape(-1, 4, order='F')[:, [0, 2, 3, 1]].reshape(-1,2,2)
    # reshuffle D
    data['simulations']['time_steps']['D'] = data['simulations']['time_steps']['D'].reshape(-1, 4, 4, order='F')[:, [[0], [2], [3], [1]], [[0, 2, 3, 1]]].reshape(-1, 2, 2, 2, 2)

    # rename keys
    data['simulations']['time_steps']['trajectory'] = data['simulations']['time_steps'].pop('traj')
    if 'bifurcMode' not in data['simulations']['time_steps']:
        print('!! Warning !! No bifurcation mode data found')
        errors.append([geom, 'No bifurcation mode data found', 12])
    else:
        data['simulations']['time_steps']['bifurcation_mode'] = data['simulations']['time_steps'].pop('bifurcMode')
    data['simulations']['trajectories']['error_flag'] = data['simulations']['trajectories'].pop('errorFlag')
    data['simulations']['time_steps']['is_bifurcation_point'] = data['simulations']['time_steps'].pop('bifurc')

    if len(data['simulations']['time_steps']['F']) == 0:
        print('!! Warning !! No simulations found')
        errors.append([geom, 'No simulations found', len(data['simulations']['trajectories']['F_final'])])
        continue

    # reshape/remove trailing dimension
    for key in ['J_final', 't_final', 'phi_final', 'computation_time', 'error_flag']:
        data['simulations']['trajectories'][key] = data['simulations']['trajectories'][key][:, 0]
    data['simulations']['time_steps']['trajectory'] = data['simulations']['time_steps']['trajectory'][:, 0]
    data['simulations']['time_steps']['is_bifurcation_point'] = data['simulations']['time_steps']['is_bifurcation_point'][:, 0]

    # reshape bifurcation mode
    if 'bifurcation_mode' in data['simulations']['time_steps']:
        data['simulations']['time_steps']['bifurcation_mode'] = data['simulations']['time_steps']['bifurcation_mode'][0] # remove singleton dimension
        for i, mode in enumerate(data['simulations']['time_steps']['bifurcation_mode']):
            if mode.shape == (0, 0):
                data['simulations']['time_steps']['bifurcation_mode'][i] = None
            else:
                data['simulations']['time_steps']['bifurcation_mode'][i] = mode.reshape(-1, 2)

    lengths = [len(data['simulations']['time_steps'][key]) for key in data['simulations']['time_steps']]
    if len(set(lengths)) != 1:
        print('!! Warning !! Not all time steps have the same length')
        print(lengths)
        errors.append([geom, 'Missing time steps', max(lengths) - min(lengths)])
        continue
    print('Nr of time steps:', len(data['simulations']['time_steps']['F']))
    print('Nr of nodes:', data['simulations']['time_steps']['microfluctuation'].shape[1])

    # ====================== LOAD REF D ======================
    path_temp = os.path.join(refD_path, geom + '.mat')
    data_D_ref = sio.loadmat(path_temp)
    # shuffle D
    D_ref = data_D_ref['Cmacro'][[[0], [2], [3], [1]], [[0, 2, 3, 1]]].reshape(2,2,2,2)
    data['simulations']['D_ref'] = D_ref

    # print('data[time_steps][trajectory]:', data['simulations']['time_steps']['trajectory'])

    # ====================== LOAD RERUN DATA ======================
    for r, asdf in enumerate([reruns, reruns2, reruns3]):
        for rerun in asdf:
            if rerun.startswith(geom):  # check if one or more of the reruns belong to this geometry
                print('Rerun found!', geom, rerun)
                rerun_found = True
                ind = int(os.path.splitext(rerun)[0].split('_')[-1]) - 1  # get index of this trajectory

                # if r == 0:  # only for the first set of reruns
                    # check if rerun indeed originally ended in an error
                if not data['simulations']['trajectories']['error_flag'][ind] and not r==2:
                    print('Rerun did not originally end in an error')
                    continue

                data_temp = {'simulations': {
                                'trajectories': {},
                                'time_steps': {}
                                }
                             }  # temporary dictionary to store rerun data

                # needed:
                # data_temp['simulations']['trajectories']['computation_time']
                # data_temp['simulations']['trajectories']['error_flag']
                # data_temp['simulations']['time_steps']: everything

                if r==0:
                    matfile = os.path.join(reruns_path, rerun)
                elif r==1:
                    matfile = os.path.join(reruns2_path, rerun)
                elif r==2:
                    matfile = os.path.join(reruns3_path, rerun)
                data_from_mat_rerun = sio.loadmat(matfile)

                # check if rerun actually has the same F
                if not np.allclose(data['simulations']['trajectories']['F_final'][ind], data_from_mat_rerun['data_sim']['F_final'][0, 0]):
                    print('F original:', data['simulations']['trajectories']['F_final'][ind])
                    print('F rerun:', data_from_mat_rerun['data_sim']['F_final'][0, 0])
                    print('Rerun does not have the same F_final')
                    continue

                # fill the data_temp dictionary
                for key in data_from_mat_rerun['data_sim'].dtype.names:
                    data_temp['simulations']['trajectories'][key] = data_from_mat_rerun['data_sim'][key][0, 0]
                for key in data_from_mat_rerun['data_ts'].dtype.names:
                    data_temp['simulations']['time_steps'][key] = data_from_mat_rerun['data_ts'][key][0, 0]

                # change from Matlab 1-indexing to Python 0-indexing
                del data_temp['simulations']['time_steps']['traj']  # will always be 1 anyways


                # reshuffle P
                data_temp['simulations']['time_steps']['P'] = data_temp['simulations']['time_steps']['P'].reshape(-1, 4, order='F')[:, [0, 2, 3, 1]].reshape(-1,2,2)
                # reshuffle D
                data_temp['simulations']['time_steps']['D'] = data_temp['simulations']['time_steps']['D'].reshape(-1, 4, 4, order='F')[:, [[0], [2], [3], [1]], [[0, 2, 3, 1]]].reshape(-1, 2, 2, 2, 2)

                # rename keys
                if 'bifurcMode' not in data_temp['simulations']['time_steps']:
                    print('!! Warning !! No bifurcation mode data_temp found')
                    errors.append([geom, 'No bifurcation mode data_temp found', 12])
                else:
                    data_temp['simulations']['time_steps']['bifurcation_mode'] = data_temp['simulations']['time_steps'].pop('bifurcMode')
                data_temp['simulations']['trajectories']['error_flag'] = data_temp['simulations']['trajectories'].pop('errorFlag')
                data_temp['simulations']['time_steps']['is_bifurcation_point'] = data_temp['simulations']['time_steps'].pop('bifurc')

                if len(data_temp['simulations']['time_steps']['F']) == 0:
                    print('!! Warning !! Rerun has 0 time steps')
                    errors.append([geom, 'Rerun has 0 time steps', 1, [ind]])
                    continue

                # reshape/remove trailing dimension
                for key in ['computation_time', 'error_flag']:
                    data_temp['simulations']['trajectories'][key] = data_temp['simulations']['trajectories'][key][:, 0]
                data_temp['simulations']['time_steps']['is_bifurcation_point'] = data_temp['simulations']['time_steps']['is_bifurcation_point'][:, 0]

                if len(data_temp['simulations']['time_steps']['F']) == 0:
                    raise ValueError('Rerun has length 0')

                data_temp['simulations']['time_steps']['trajectory'] = np.array([ind]*len(data_temp['simulations']['time_steps']['F']))  # set trajectory to the index of the rerun

                # reshape bifurcation mode
                if 'bifurcation_mode' in data_temp['simulations']['time_steps']:
                    data_temp['simulations']['time_steps']['bifurcation_mode'] = data_temp['simulations']['time_steps']['bifurcation_mode'][0] # remove singleton dimension
                    for i, mode in enumerate(data_temp['simulations']['time_steps']['bifurcation_mode']):
                        if mode.shape == (0, 0):
                            data_temp['simulations']['time_steps']['bifurcation_mode'][i] = None
                        else:
                            data_temp['simulations']['time_steps']['bifurcation_mode'][i] = mode.reshape(-1, 2)

                lengths = [len(data_temp['simulations']['time_steps'][key]) for key in data_temp['simulations']['time_steps']]
                if len(set(lengths)) != 1:
                    print('!! Warning !! Not all time steps have the same length')
                    print(lengths)
                    errors.append([geom, 'Missing time steps', max(lengths) - min(lengths)])
                    continue

                # insert new data into the original data
                inds_replace = np.where(data['simulations']['time_steps']['trajectory'] == ind)[0]
                if len(inds_replace) > 0:
                    start_ind = inds_replace[0]
                    end_ind = inds_replace[-1] + 1
                else:  # if the original trajectory had 0 time steps, you can't find the start and end indices
                    # find last index where ind < trajectory
                    inds_temp = np.where(data['simulations']['time_steps']['trajectory'] < ind)[0]
                    if len(inds_temp) > 0:
                        start_ind = inds_temp[-1] + 1
                        end_ind = start_ind
                    else:  # if there are no trajectories < ind, it must be the first trajectory
                        start_ind = 0
                        end_ind = 0

                for key in data['simulations']['time_steps']:
                    data['simulations']['time_steps'][key] = np.concatenate([data['simulations']['time_steps'][key][:start_ind], data_temp['simulations']['time_steps'][key], data['simulations']['time_steps'][key][end_ind:]])  # replace the data of the rerun

                data['simulations']['trajectories']['error_flag'][ind] = data_temp['simulations']['trajectories']['error_flag'][0]  # replace the error flag
                data['simulations']['trajectories']['computation_time'][ind] = data_temp['simulations']['trajectories']['computation_time'][0]  # replace the computation time

    # Remove empty trajectories
    tr = data['simulations']['time_steps']['trajectory']
    n = len(data['simulations']['trajectories']['F_final'])
    tr2, inv = np.unique(tr, return_inverse=True)
    counts = np.bincount(tr, minlength=n)
    if len(tr2) != n:
        print('!! Warning !! Not all trajectories are present')
        errors.append([geom, 'Trajectories of length 0 in data', n-len(tr2), np.where(counts == 0)[0]])
        # remove trajectories of length 0
        for key in data['simulations']['trajectories']:
            data['simulations']['trajectories'][key] = data['simulations']['trajectories'][key][counts > 0]
        data['simulations']['time_steps']['trajectory'] = inv

    # ====================== LOAD SPECIAL NODES DATA ======================
    matfile = os.path.join(results_path, geom + '_specialnodes.mat')
    data_from_mat_specialnodes = sio.loadmat(matfile)
    # change from Matlab 1-indexing to Python 0-indexing
    data['mesh']['RVE']['fixed_node'] = data_from_mat_specialnodes['fixed_node'][0,0]-1
    data['mesh']['RVE']['source_nodes'] = data_from_mat_specialnodes['source_nodes'][:, 0]-1
    data['mesh']['RVE']['image_nodes'] = data_from_mat_specialnodes['image_nodes'][:, 0]-1

    # ====================== LOAD GEOMETRY DATA (.mat) ======================
    # load data that was sent to matlab
    path = os.path.join(geometries_path, geom, geom + '_00.mat')
    data_to_mat = sio.loadmat(path)

    data['mesh']['RVE']['p'] = data_to_mat['p']
    data['mesh']['RVE']['t'] = data_to_mat['t']
    data['mesh']['RVE']['boundary_inds'] = data_to_mat['boundary_inds']
    data['mesh']['RVE']['inds_per_fd'] = data_to_mat['inds_per_fd']
    data['mesh']['RVE']['volume_fraction'] = data_to_mat['volume_fraction'][0,0]

    # check dtype
    if data['mesh']['RVE']['boundary_inds'].dtype == object:
        b = data['mesh']['RVE']['boundary_inds'][0]
        data['mesh']['RVE']['boundary_inds'] = [b2[0].tolist() for b2 in b]
    else:
        data['mesh']['RVE']['boundary_inds'] = data['mesh']['RVE']['boundary_inds'].tolist()
    # print(len(data['mesh']['RVE']['boundary_inds']), len(data['mesh']['RVE']['boundary_inds'][0]), type(data['mesh']['RVE']['boundary_inds']), type(data['mesh']['RVE']['boundary_inds'][0]))

    # ====================== LOAD GEOMETRY DATA (.pkl) ======================
    # load data from .pkl file from geometry generation
    with open(os.path.join(geometries_path, geom, geom + '_00.pkl'), 'rb') as f:
        data_pkl = pickle.load(f)

    # add unit cell data
    for key, value in data_pkl['uc'].items():
        data['geometry']['unit_cell'][key] = value

    # rename lattice vectors to lattice_vectors
    data['geometry']['unit_cell']['lattice_vectors'] = data['geometry']['unit_cell'].pop('lattice vectors')

    # delete equiv_nodes, is equivalent to periodic_nodes
    del data['geometry']['unit_cell']['equiv_nodes']

    # add fundamental domain data
    for key, value in data_pkl['fd'].items():
        data['geometry']['fundamental_domain'][key] = value

    # delete unnecessary keys
    del data['geometry']['fundamental_domain']['points']
    del data['geometry']['fundamental_domain']['bound_inds']
    del data['geometry']['fundamental_domain']['edges']
    del data['geometry']['fundamental_domain']['n_points']
    del data['geometry']['fundamental_domain']['n_edges']

    # add unique holes data
    data['geometry']['unique_faces'] = data_pkl['unique_holes']

    uf = data['geometry']['unique_faces']
    for elem in uf:
        if 'equiv_holes' in elem:
            elem['equiv_faces'] = elem.pop('equiv_holes')
        if 'all_holes' in elem:
            elem['all_faces'] = elem.pop('all_holes')

    for elem in data['geometry']['unique_faces']:
        # to do: might throw an error if bisectors_x or splint_point_dists are not defined
        if 'bisectors_x' in elem:
            elem['bisectors_x'] = np.array(elem['bisectors_x'])
        if 'spline_point_dists' in elem:
            elem['spline_point_dists'] = np.array(elem['spline_point_dists'])

    data['mesh']['fundamental_domain']['p'] = data_pkl['p']
    data['mesh']['fundamental_domain']['t'] = data_pkl['t']

    data['mesh']['unit_cell']['p'] = data_pkl['p_all']
    data['mesh']['unit_cell']['t'] = data_pkl['t_all']
    data['mesh']['unit_cell']['boundary_inds'] = data_pkl['boundary_inds']

    # ====================== CREATE convenient quantities ======================
    # Calculate position of points at each time step
    F = data['simulations']['time_steps']['F']  # shape (n_time_steps, 2, 2)
    x_0 = data['mesh']['RVE']['p']  # shape (n_points, 2)
    w = data['simulations']['time_steps']['microfluctuation']  # shape (n_time_steps, n_points, 2)
    x = np.einsum('ijk,lk->ilj', F, x_0) + w
    data['simulations']['time_steps']['x'] = x

    # Create edges
    # turn the elements into edges and deduplicate them
    t = data['mesh']['RVE']['t']
    edges = np.vstack((t[:, [0, 3]],
                    t[:, [3, 1]],
                    t[:, [1, 4]],
                    t[:, [4, 2]],
                    t[:, [2, 5]],
                    t[:, [5, 0]]))
    edges2 = np.sort(edges, axis=1)
    edges3, counts = np.unique(edges2, axis=0, return_counts=True)
    data['mesh']['RVE']['edges'] = edges3

    # find all edges that are not shared by two triangles: boundary edges
    # (either boundary of the RVE of boundary of a hole)
    # for each edge in the boundary, check if both nodes are in the same array in boundary_inds
    # if so, it is a unit cell boundary edge
    b_edges = edges3[counts!=2]
    bools = np.zeros(len(b_edges), dtype=bool)
    # print(len(data['mesh']['RVE']['boundary_inds']), len(data['mesh']['RVE']['boundary_inds'][0]))
    for b in data['mesh']['RVE']['boundary_inds']:
        bools[np.isin(b_edges[:, 0], b)*np.isin(b_edges[:, 1], b)] = True
    bools2 = np.zeros(len(edges3), dtype=bool)
    bools2[counts!=2] = bools
    data['mesh']['RVE']['boundary_edges_inds'] = np.where(bools2)[0]

    # select hole boundary edges
    # (edges that are boundary edges but not unit cell boundary edges)
    bools3 = np.zeros(len(edges3), dtype=bool)
    bools3[counts!=2] = True  # all boundary edges
    bools3[bools2] = False     # remove unit cell boundary edges, leaving only hole boundary edges  #!! edge case: if a hole boundary edge is also a unit cell boundary edge, this will go wrong!!
    data['mesh']['RVE']['hole_boundary_edges_inds'] = np.where(bools3)[0]

    # ====================== REMOVE CONTACT ======================
    edges = data['mesh']['RVE']['edges']
    hb_inds = data['mesh']['RVE']['hole_boundary_edges_inds']
    hb_edges = edges[hb_inds]
    # only use coordinates of hole boundary nodes and edges at the hole boundary
    hb_nodes, hb_edges = np.unique(hb_edges, return_inverse=True)
    hb_edges = hb_edges.reshape(-1, 2)
    hb_x = data['simulations']['time_steps']['x'][:, hb_nodes]

    # specify end and start of each trajectory
    traj = data['simulations']['time_steps']['trajectory']
    n_trajs = np.max(traj)+1
    start_inds = np.concatenate(([0], np.where(traj[:-1] != traj[1:])[0]+1))
    end_inds = np.concatenate((start_inds[1:], [len(traj)]))

    ends_in_contact = np.zeros(n_trajs, dtype=bool)
    i = 0
    while i < len(traj):
        # print(i)
        traj_i = traj[i]

        # Get the positions of the nodes
        x = data['simulations']['time_steps']['x'][i]
        hb_x_i = hb_x[i]

        # check if a pair of edges intersects
        cr = mf.crossing_edges(hb_x_i, hb_edges)

        if False:
            # plot all nodes
            plt.figure()
            plt.scatter(*(hb_x_i.T), s=2, c='tab:blue')

            # plot boundary edges
            x, y = hb_x_i.T[:, hb_edges]
            x, y = x.T, y.T
            plt.plot(x, y, c='tab:blue', alpha=1, zorder=-1)

            plt.gca().set_aspect('equal')
            plt.grid()

        if cr.size > 0:
            ends_in_contact[traj_i] = True
            # print(f'i={i}, traj={traj[i]}, contact detected')

            if False:
                # plot all nodes
                plt.figure()
                plt.scatter(*(hb_x_i.T), s=2, c='tab:blue')

                # plot boundary edges
                x, y = hb_x_i.T[:, hb_edges]
                x, y = x.T, y.T
                plt.plot(x, y, c='tab:blue', alpha=1, zorder=-1)

                # plot nodes of crossing edges
                nodes = hb_edges[cr].reshape(-1, 4)
                nodes2 = np.unique(nodes)
                plt.scatter(*(hb_x_i[nodes2].T), s=2, c='tab:orange')

                # plot crossing edges
                x, y = hb_x_i.T[:, hb_edges[cr].reshape(-1, 2)]
                x, y = x.T, y.T
                plt.plot(x, y, c='tab:orange', alpha=1, zorder=-1)

                plt.gca().set_aspect('equal')
                plt.grid()
                plt.savefig(os.path.join(final_path, geom + f'_contact_traj{traj_i}.png'))
                plt.close()

            end_inds[traj_i] = i # this trajectory ends early
            if traj_i == n_trajs-1: # last trajectory, break for loop
                break
            else: # skip rest of the time steps of this trajectory, go to start of next trajectory
                if i > start_inds[traj_i+1]:
                    raise ValueError('i should go back to a previous time step, which makes no sense')
                i = start_inds[traj_i+1]

        else:
            i += 1


    inds_to_keep = np.concatenate([np.arange(i,j) for i,j in zip(start_inds, end_inds)])
    # print(f'start_inds: \t{start_inds}')
    # print(f'end_inds: \t{end_inds}')
    print(f'Trajectory lengths: {end_inds-start_inds}')
    # print(f'ends_in_contact: \t{ends_in_contact.astype(int)}')
    # print(f'ErrorFlag: \t{data["simulations"]["error_flag"]}')
    error2 = data['simulations']['trajectories']['error_flag']
    error2[ends_in_contact] = False
    print(f'Error before contact: {error2}')

    if np.any(end_inds - start_inds <= 2):
        print('!! Warning !! One or more trajectories have length <= 2')
        errors.append([geom, 'Trajectories of length <= 2 after removing contact', np.sum(end_inds-start_inds <= 2), np.where(end_inds-start_inds <= 2)[0]])
    if np.any(error2):

        print('!! Warning !! One or more trajectories still end in an error when contact is removed')
        n_err = error2.sum()
        errors.append([geom, 'Error remains after removing contact', n_err, np.where(error2)[0]])

        # if True:
        #     # plot all nodes
        #     plt.figure()
        #     plt.suptitle(geom + ' - trajectory ending in an error')
        #     # of the first trajectory ending in an error, take last time step
        #     ind_temp = end_inds[error2][0] - 1
        #     hb_x_i = hb_x[ind_temp]
        #     plt.scatter(*(hb_x_i.T), s=2, c='tab:blue')

        #     # plot boundary edges
        #     x, y = hb_x_i.T[:, hb_edges]
        #     x, y = x.T, y.T
        #     plt.plot(x, y, c='tab:blue', alpha=1, zorder=-1)

        #     plt.gca().set_aspect('equal')
        #     plt.grid()
        #     plt.savefig(os.path.join(final_path, geom + f'_error_traj{np.where(error2)[0][0]}.png'))
        #     plt.close()

    for key in data['simulations']['time_steps']:
        data['simulations']['time_steps'][key] = data['simulations']['time_steps'][key][inds_to_keep]

    data['simulations']['trajectories']['error_flag'] = error2.astype(bool)
    data['simulations']['trajectories']['ends_in_contact'] = ends_in_contact.astype(bool)

    # sum up is_bifurcation_point per trajectory
    inds_slice = np.nonzero(np.diff(data['simulations']['time_steps']['trajectory']))[0] + 1
    inds_slice = np.insert(inds_slice, 0, 0)  # add zero at the beginning
    data['simulations']['trajectories']['contains_bifurcation'] = np.add.reduceat(data['simulations']['time_steps']['is_bifurcation_point'], inds_slice).astype(bool)

    # ====================== SAVE DATA ======================
    with open(os.path.join(final_path, geom + '.pkl'), 'wb') as f:
        pickle.dump(data, f)

path = r'E:\OneDrive - TU Eindhoven\Werk\python\wallpaper_dataset\data\dataset1\errors.pkl'
with open(path, 'wb') as f:
    pickle.dump(errors, f)


In [ ]:

path = r'E:\OneDrive - TU Eindhoven\Werk\python\wallpaper_dataset\data\dataset1\errors.pkl'
with open(path, 'wb') as f:
    pickle.dump(errors, f)


In [ ]:
# pretty print all data: keys, shapes, dtype
for key, value in data.items():
    if isinstance(value, dict):
        print(key)
        for key2, value2 in value.items():
            if isinstance(value2, dict):
                print(f'    {key2}')
                for key3, value3 in value2.items():
                    if isinstance(value3, dict):
                        print(f'        {key3}')
                        for key4, value4 in value3.items():
                            print(f'            {key4}: {value4.shape} {value4.dtype}')
                    elif isinstance(value3, np.ndarray):
                        print(f'        {key3}: {value3.shape} {value3.dtype}')
                    else:
                        print(f'        {key3}: {value3}')
            elif isinstance(value2, np.ndarray):
                print(f'    {key2}: {value2.shape} {value2.dtype}')
            else:
                print(f'    {key2}: {value2}')
    elif isinstance(value, np.ndarray):
        print(f'{key}: {value.shape} {value.dtype}')
    else:
        print(f'{key}: {value}')

# Create .png for each geometry

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import mesh_funcs as mf

save_dir = r'data\dataset1\final_dataset\images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

path = r'data\dataset1\final_dataset'

In [ ]:
# set font size bigger
plt.rcParams.update({'font.size': 20})

In [ ]:
%matplotlib qt

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches

for file in os.listdir(path):
    if file.endswith('.pkl'):
        with open(os.path.join(path, file), 'rb') as f:
            data = pickle.load(f)

        group = data['geometry']['fundamental_domain']['group']
        shape = data['geometry']['fundamental_domain']['shape']
        uc_bounds = data['geometry']['unit_cell']['bounds']
        fd_bounds = data['geometry']['fundamental_domain']['bounds']

        # Plot unit cell 2×2
        fig, ax = plt.subplots(figsize=(14, 14))
        ax.set_title(f'{group} ({shape})')
        fig.patch.set_facecolor("None")

        # plot black boundary around unit cell
        temp = np.stack(uc_bounds)
        temp[:, 1] += temp[:, 0]
        ax.plot(*temp.T, c='black')
        # plot black boundary around fundamental domain
        temp = np.stack(fd_bounds)
        temp[:, 1] += temp[:, 0]
        ax.plot(*temp.T, c='black')

        uc_corners = data['geometry']['unit_cell']['corners']
        fd_corners = data['geometry']['fundamental_domain']['corners']
        # background fundamental domain shape
        ax.fill(*uc_corners.T, c='lightgray', alpha=0.5, zorder=-10, antialiased=True, snap=False)

        # background fundamental domain shape
        ax.fill(*fd_corners.T, c='lightgray', alpha=0.5, zorder=-10, antialiased=True, snap=False)

        all_mesh_points = data['mesh']['unit_cell']['p']
        vecs = data['geometry']['unit_cell']['lattice_vectors']
        all_elements = data['mesh']['unit_cell']['t']

        # loop over vecs[0] translations
        for i in range(2):
            # loop over vecs[1] translations
            for j in range(2):
                points_temp = mf.translate_points(all_mesh_points, i*vecs[0]+j*vecs[1])

                # plot filled triangles
                temp = points_temp[all_elements]
                temp = np.transpose(temp, axes=[0,2,1])
                temp = temp.reshape(-1, temp.shape[-1])
                temp = temp[..., [0,3,1,4,2,5]]
                if i == 0 and j == 0:
                    ax.fill(*temp,
                            # antialiased=True,
                            # snap=False,
                            linewidth=0)
                else:
                    ax.fill(*temp, c='tab:orange', #antialiased=True,
                            # snap=False,
                            linewidth=0, edgecolor='tab:orange')

                # ax.scatter(*points_temp.T, alpha=0.5, s=50)  #, c='tab:orange')
                # x, y = np.transpose(points_temp[uc["edges"].T], axes=[2,0,1])
                # edges0 = ax.plot(x, y, alpha=0.3, c='tab:red')

        ax.set_aspect('equal')

        # # Rasterize the patches
        # for artist in ax.get_children():
        #     if isinstance(artist, patches.Patch):
        #         artist.set_rasterized(True)
        # fig.set_rasterized(True)

        fig.savefig(os.path.join(save_dir, file[:-4] + '.png'),bbox_inches='tight', dpi=300)
        plt.close()


        # break
